In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/south-african-language-identification/sample_submission.csv
/kaggle/input/south-african-language-identification/test_set.csv
/kaggle/input/south-african-language-identification/train_set.csv


# 1. Importing packages

In [2]:
# Packages for data analysis
import pandas as pd
import numpy as np
import time

# Packages for visualizations
import seaborn as sns
import matplotlib.style as style

# Packages for preprocessing
import nltk
import string
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer

# Packages for training models
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import xgboost as xgb

# Model Evaluation Packages
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.metrics import make_scorer

import matplotlib.pyplot as plt
%matplotlib inline

# Style
sns.set(font_scale=1.5)
style.use('seaborn-pastel')
style.use('seaborn-poster')

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# 2. Loading of dataset

In [4]:
# importing the dataset
train = pd.read_csv('../input/south-african-language-identification/train_set.csv')
test = pd.read_csv('../input/south-african-language-identification/test_set.csv')
sample_submission = pd.read_csv('../input/south-african-language-identification/sample_submission.csv')


In [5]:
train.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [6]:
test.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


<h1 style="font-size:2em;color:#2467C0">Exploring Data</h1>

We will start our data exploration by generating simple statistics of the data. 
<br>
Let us look at what the train data columns are, using a pandas attribute called "info", value counts...

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [8]:
train.lang_id.value_counts()

xho    3000
eng    3000
nso    3000
ven    3000
tsn    3000
nbl    3000
zul    3000
ssw    3000
tso    3000
sot    3000
afr    3000
Name: lang_id, dtype: int64

<strong><h1 style="font-size:1.1em;color:#1F618D">Training Data</h1></strong>

In [9]:
print('Shape of the dataset: {}\n'.format(train.shape))
print('Total Number of unique tweets: {}\n'.format(len(set(train['text']))))
print('Total Number of missing values:\n{}\n\n'.format(train.isnull().sum()))

Shape of the dataset: (33000, 2)

Total Number of unique tweets: 29948

Total Number of missing values:
lang_id    0
text       0
dtype: int64




<strong><h1 style="font-size:1.1em;color:#1F618D">Testing Data</h1></strong>

In [10]:
# Test Data
print('Shape of the dataset: {}\n'.format(test.shape))
print('Total Number of unique tweets: {}\n'.format(len(set(test['text']))))
print('Total Number of missing values:\n{}\n' .format(test.isnull().sum()))

Shape of the dataset: (5682, 2)

Total Number of unique tweets: 5459

Total Number of missing values:
index    0
text     0
dtype: int64



 <h1 style="font-size:2em;color:#00000">3. Data Preprocessing</h1>

In [11]:
def clean_text(text):
    """
    This function uses regular expressions to remove html characters,
    punctuation, numbers and any extra white space from each text
    and then converts them to lowercase.

    Input:
    text: original text
          datatype: string

    Output:
    texts: modified text
           datatype: string
    """
    # replace the html characters with " "
    text=re.sub('<.*?>', ' ', text)
#     Removal of numbers
#    text = re.sub(r'\d+', ' ', text)
    # will replace newline with space
    text = re.sub("\n"," ",text)
    # will convert to lower case
    text = text.lower()
    # will split and join the words
    text=' '.join(text.split())
    return text

In [12]:
# Application of the function to clean the tweets
train['text'] = train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [13]:
# Replace '.txt' with 'text file'
train["text"] = train["text"].str.replace(".txt", " text file")
test["text"] = test["text"].str.replace(".txt", " text file")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


# 4. Feature Engineering 

### 4.1 Splitting out X (indepedent) and Y (target/dependent) variables

In [14]:
X = train['text']
y = train['lang_id']

### 4.2 Splitting of Training and Validation Sets

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10)

# 5. Model Building 

### set up classifier

In [16]:
"""
Note: Some classifiers were commented out because
they run for a very long time, 
"""
classifiers = [LinearSVC(random_state=42),
               # SVC(),
               # tree.DecisionTreeClassifier(),
               # RandomForestClassifier(n_estimators=100, max_depth=2,
               #                      random_state=0, class_weight="balanced"),
               # MLPClassifier(alpha=1e-5,
               #              hidden_layer_sizes=(5, 2),
               #              random_state=42),
               LogisticRegression(random_state=42,
                                  multi_class='ovr',
                                  n_jobs=1,
                                  C=1e5,
                                  max_iter=4000),
               KNeighborsClassifier(n_neighbors=5),
               MultinomialNB(),
               ComplementNB(),
               SGDClassifier(loss='hinge',
                             penalty='l2',
                             alpha=1e-3,
                             random_state=42,
                             max_iter=5,
                             tol=None)
               # GradientBoostingClassifier()
               # xgb.XGBClassifier(learning_rate=0.1,
               #                  n_estimators=1000,
               #                  max_depth=5,
               #                  min_child_weight=1,
               #                  gamma=0,
               #                  subsample=0.8,
               #                  colsample_bytree=0.8,
               #                  nthread=4,
               #                  seed=27)
               ]

##### creating funstion for model building 

In [17]:
def models_building(classifiers, X_train, y_train, X_val, y_val):
    """
    This function takes in a list of classifiers
    and both the train and validation sets
    and return a summary of F1-score and
    processing time as a dataframe

    Input:
    classifiers: a list of classifiers to train
                 datatype: list
    X_train: independent variable for training
             datatype: series
    y_train: dependent variable for training
             datatype: series
    X_val: independent variable for validation
           datatype: series
    y_val: dependent variable for validation
           datatype: series

    Output:
    model_summary: F1 Score for all the classifiers
                   datatype: dataframe
    """

    models_summary = {}

    # Pipeline to balance the classses and then to build the model
    for clf in classifiers:
        clf_text = Pipeline([('tfidf', TfidfVectorizer(min_df=1,
                                                       max_df=0.9,
                                                       ngram_range=(1, 2))),
                             ('clf', clf)])

        # Logging the Execution Time for each model
        start_time = time.time()
        clf_text.fit(X_train, y_train)
        predictions = clf_text.predict(X_val)
        run_time = time.time()-start_time

        # Output for each model
        models_summary[clf.__class__.__name__] = {
            'F1-Macro': metrics.f1_score(y_val,
                                         predictions,
                                         average='macro'),
            'F1-Accuracy': metrics.f1_score(y_val, predictions,
                                            average='micro'),
            'F1-Weighted': metrics.f1_score(y_val,
                                            predictions,
                                            average='weighted'),
            'Execution Time': run_time}

    return pd.DataFrame.from_dict(models_summary, orient='index')

##### Excute the classifiers 

In [18]:
classifiers_df = models_building(classifiers, X_train, y_train, X_val, y_val)
ordered_df = classifiers_df.sort_values('F1-Macro', ascending=False)
ordered_df

,F1-Macro,F1-Accuracy,F1-Weighted,Execution Time
MultinomialNB,0.999110,0.999091,0.999090,6.719995
LinearSVC,0.997891,0.997879,0.997877,9.106623
LogisticRegression,0.997891,0.997879,0.997877,218.463656
ComplementNB,0.997587,0.997576,0.997572,6.798497
SGDClassifier,0.989924,0.990000,0.989957,7.858136
KNeighborsClassifier,0.969532,0.969697,0.969443,10.290909



5.1.2 Comparing Classification Methods

The most performing is the Multinomial Naive Bayes with F1-Macro of 99.9% and accuracy of 99.9% while closely followed by Complement Naive Bayes, Logistic Regression, Linear Support Vector Classifier, Support Vector Machine etc.

We will proceed with the first two algorithms (to see which will come out better) by applying hyperparameter tunining, as they are the most performing models and considering their execution time.

## 5.2 Hyperparameter tuning 

In [19]:
# Refining the train-test split for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01)

##### 5.2.1 Multinomial naive baiyes

In [20]:
# Creating a pipeline for the gridsearch
param_grid = {'alpha': [0.1, 1, 5, 10]}  # setting parameter grid

tuned_mnb = Pipeline([('tfidf', TfidfVectorizer(min_df=2,
                                                max_df=0.9,
                                                ngram_range=(1, 2))),
                      ('mnb', GridSearchCV(MultinomialNB(),
                                           param_grid=param_grid,
                                           cv=5,
                                           n_jobs=-1,
                                           scoring='f1_weighted'))
                      ])

tuned_mnb.fit(X_train, y_train)  # Fitting the model

y_pred_mnb = tuned_mnb.predict(X_val)  # predicting the fit on validation set

print(classification_report(y_val, y_pred_mnb))

              precision    recall  f1-score   support

         afr       1.00      1.00      1.00        38
         eng       1.00      1.00      1.00        27
         nbl       1.00      1.00      1.00        35
         nso       1.00      1.00      1.00        23
         sot       1.00      1.00      1.00        32
         ssw       1.00      1.00      1.00        34
         tsn       1.00      1.00      1.00        34
         tso       1.00      1.00      1.00        28
         ven       1.00      1.00      1.00        27
         xho       1.00      1.00      1.00        24
         zul       1.00      1.00      1.00        28

    accuracy                           1.00       330
   macro avg       1.00      1.00      1.00       330
weighted avg       1.00      1.00      1.00       330



#### 5.3 Final Submission 

In [21]:
submission_df = pd.DataFrame(test['index'])
submission_df['lang_id'] = tuned_mnb.predict(test['text'])
submission_df.to_csv('submission_tuned_multinomial_NB.csv', index=False)

## 6. Conclusion


Several algorithms were tried and Multinomial Naive Bayes classifier was the most performing. It performed very well on the training and validation datasets with an accuracy score of over 99% and F1 Macro score of over 99%. After testing the fitted model on the held-out/unseen dataset, it was able to predict the classes of languages with an F1 Score of about 97%.

## 7. References

1. Hyperparameters and Model Validation (An overview of classification model hyperparameters, hyperparameter tuning, and model validation) - Explore Data Science Academy
2. https://scikit-learn.org/stable/modules/grid_search.html

